In [ ]:
%pip install azure-ai-evaluation --upgrade
%pip install promptflow-azure
%pip install azure-identity

In [ ]:
import os
from typing import Any, Dict, List, Optional
import json
from pathlib import Path

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import GroundednessEvaluator
from azure.ai.evaluation.simulator import Simulator
from openai import AzureOpenAI
import importlib.resources as pkg_resources
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [ ]:
os.environ["AZURE_DEPLOYMENT_NAME"] = "Ravi-OpenAI-Inst-032523"
os.environ["AZURE_ENDPOINT"] = "https://ravi-openai-inst-032523.openai.azure.com/"
os.environ["AZURE_API_VERSION"] = "gpt-35-turbo"
os.environ["AZURE_SUBSCRIPTION_ID"] = "a234e53c-b063-4702-bcf4-9457e53c7af2"
os.environ["AZURE_RESOURCE_GROUP"] = "Ravi_OpenAI032523"
os.environ["AZURE_PROJECT_NAME"] = "safety_eval"

In [ ]:
project_scope = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("RESOURCE_GROUP"),
    "project_name": os.environ.get("PROJECT_NAME"),
}

model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "azure_deployment": os.environ.get("AZURE_DEPLOYMENT_NAME"),
}

In [ ]:
resource_name = "grounding.json"
package = "azure.ai.evaluation.simulator._data_sources"
conversation_turns = []

with pkg_resources.path(package, resource_name) as grounding_file, Path.open(grounding_file, "r") as file:
    data = json.load(file)

for item in data:
    conversation_turns.append([item])
    if len(conversation_turns) == 2:
        break

In [ ]:
def example_application_response(query: str, context: str) -> str:
    deployment = os.environ.get("AZURE_DEPLOYMENT_NAME")
    endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

    # Get a client handle for the AOAI model
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_version=os.environ.get("AZURE_API_VERSION"),
        azure_ad_token_provider=token_provider,
    )

    # Prepare the messages
    messages = [
        {
            "role": "system",
            "content": f"You are a user assistant who helps answer questions based on some context.\n\nContext: '{context}'",
        },
        {"role": "user", "content": query},
    ]
    # Call the model
    completion = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )

    message = completion.to_dict()["choices"][0]["message"]
    if isinstance(message, dict):
        message = message["content"]
    return message

In [ ]:
async def custom_simulator_callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Optional[str] = None,
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    application_input = latest_message["content"]
    context = latest_message.get("context", None)
    # call your endpoint or ai application here
    response = example_application_response(query=application_input, context=context)
    # we are formatting the response to follow the openAI chat protocol format
    message = {
        "content": response,
        "role": "assistant",
        "context": context,
    }
    messages["messages"].append(message)
    return {"messages": messages["messages"], "stream": stream, "session_state": session_state, "context": context}

In [ ]:
custom_simulator = Simulator(model_config=model_config)

In [ ]:
outputs = await custom_simulator(
    target=custom_simulator_callback,
    conversation_turns=conversation_turns,
    max_conversation_turns=1,
    concurrent_async_tasks=10,
)

In [ ]:
output_file = "ground_sim_output.jsonl"
with Path.open(output_file, "w") as file:
    for output in outputs:
        file.write(output.to_eval_qr_json_lines())

In [ ]:
groundedness_evaluator = GroundednessEvaluator(model_config=model_config)
eval_output = evaluate(
    data=output_file,
    evaluators={
        "groundedness": groundedness_evaluator,
    },
    azure_ai_project=project_scope,
)
print(eval_output)